<a href="https://colab.research.google.com/github/HadiAlizadeh1/mci-chatbot/blob/main/rasa_Farsi_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **In the name of Allah**


**In this task we are going to create a simple rasa Farsi chat bot**

In [1]:
!pip install rasa==1.10.3 #instaling rasa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install -U tables==3.6.1
!pip install -U ipython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!python -m spacy download en_core_web_sm # download en_core_web_sm data

2022-11-16 20:32:38.697358: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-11-16 20:32:38.698830: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-11-16 20:32:38.698862: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.2 MB

In [4]:
!pip install nest_asyncio==1.3.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import os
import rasa
import nest_asyncio
nest_asyncio.apply()
print('event loop is ready')

event loop is ready


In [6]:
from rasa.cli.scaffold import create_initial_project

In [7]:
project = "Farsi_bot"  # defining rasa project name
create_initial_project(project) # creat rasa project

In [8]:
# move into project directory and show files
os.chdir(project)
print(os.listdir("."))

['credentials.yml', 'actions.py', 'endpoints.yml', 'data', 'tests', 'config.yml', 'domain.yml', '__init__.py', '__pycache__']


## In this section we are going to change rasa chatbot language and create Farsi chatbot.

##NLU data:
For an assistant to recognize what a user is saying no matter how the user phrases their message, we need to provide example messages the assistant can learn from. We group these examples according to the idea or the goal the message is expressing, which is also called the intent. In the code block on the right, we have added an intent called greet, which contains example messages like “Hi”, “Hey”, and “good morning”.

In [26]:
nlu_md = """
nlu:
## intent:greet
- سلام
-سلام علیکم
-به به سلام


## intent:goodbye
-خداحافظ
-خدانگهدار
-میبینمت
-بای

## intent:affirm
-بله
-درست میگی
-آره

## intent:deny
-نه
-اصلا
-فکر نکنم

## intent:mood_great
-خوبم
-مرسی
-تشکر

## intent:mood_unhappy
-بد
-خوب نیستم

## intent:bot_challenge
- آیا شما یک ربات هستید؟
- آیا شما یک انسان هستید؟
- آیا من با یک ربات صحبت می کنم؟
-  آیا من با یک انسان صحبت می کنم؟
"""

%store nlu_md >/content/Farsi_bot/data/nlu.md

Writing 'nlu_md' (str) to file '/content/Farsi_bot/data/nlu.md'.


##Stories are example conversations that train an assistant to respond correctly depending on what the user has said previously in the conversation. The story format shows the intent of the user message followed by the assistant’s action or response.

In [19]:
stories_md = """
## happy path
* greet
  - utter_greet
* mood_great
  - utter_happy

## sad path 1
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* affirm
  - utter_happy

## sad path 2
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* deny
  - utter_goodbye

## say goodbye
* goodbye
  - utter_goodbye

## bot challenge
* bot_challenge
  - utter_iamabot

"""

%store stories_md >/content/Farsi_bot/data/stories.md

Writing 'stories_md' (str) to file '/content/Farsi_bot/data/stories.md'.


In [52]:
domain_yml="""
intents:
  - greet
  - goodbye
  - affirm
  - deny
  - mood_great
  - mood_unhappy
  - bot_challenge

responses:
  utter_greet:
  - text: "سلام ، چه خبر،خوبی؟"

  utter_cheer_up:
  - text: "اینجا چیزی است که تو رو خوشحال میکنه"
    image: "https://i.imgur.com/nGF1K8f.jpg"

  utter_did_that_help:
  - text: "خوب بود؟"

  utter_happy:
  - text: "عالیه"

  utter_goodbye:
  - text: "خدا نگهدارت"

  utter_iamabot:
  - text: "من یک بات هستم "

session_config:
  session_expiration_time: 60
  carry_over_slots_to_new_session: true
"""
%store domain_yml >/content/Farsi_bot/domain.yml

Writing 'domain_yml' (str) to file '/content/Farsi_bot/domain.yml'.


In [53]:
config = "config.yml"
training_files = "data/"
domain = "domain.yml"
output = "models/"
print(config, training_files, domain, output)

config.yml data/ domain.yml models/


In [54]:
model_path = rasa.train(domain, config, [training_files], output) # training rasa chat bot
print(model_path) # print the path of model

Processed Story Blocks: 100%|██████████| 5/5 [00:00<00:00, 104.60it/s, # trackers=20]

Training Core model...



Processed trackers: 100%|██████████| 5/5 [00:00<00:00, 116.23it/s, # actions=16]
Processed actions: 16it [00:00, 365.37it/s, # examples=16]
Epochs: 100%|██████████| 100/100 [00:19<00:00,  5.17it/s, t_loss=0.084, loss=0.013, acc=1.000]


Core model training completed.
Training NLU model...


/usr/local/lib/python3.7/dist-packages/rasa/utils/common.py:363: UserWarning: You specified 'DIET' to train entities, but no entities are present in the training data. Skip training of entities.
Epochs: 100%|██████████| 100/100 [00:22<00:00,  4.40it/s, t_loss=1.227, i_loss=0.088, i_acc=1.000]


NLU model training completed.
Your Rasa model is trained and saved at '/content/Farsi_bot/models/20221116-234748.tar.gz'.
models/20221116-234748.tar.gz


In [55]:
from rasa.jupyter import chat # runing rasa in jupyter
endpoints = 'endpoints.yml'
chat(model_path, endpoints)

Your bot is ready to talk! Type your messages here or send '/stop'.
سلام
سلام ، چه خبر،خوبی؟
خوب نیستم
اینجا چیزی است که تو رو خوشحال میکنه


خوب بود؟
بله
عالیه
خداحافظ
خدا نگهدارت
/stop


In [ ]:
## Hadi_Alizadeh